In [1]:
import os
import gc
import pickle
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm.notebook import tqdm
from sklearn.cluster import KMeans
import cudf

pd.options.display.max_rows = None
pd.options.display.max_columns = None

%matplotlib inline

In [2]:
df = pd.read_csv('features.csv')

In [3]:
df.head()

,customer_ID,S_2,D_39,D_41,D_42,D_43,D_44,D_45,D_46,D_47,D_48,D_49,D_50,D_51,D_52,D_53,D_54,D_55,D_56,D_58,D_59,D_60,D_61,D_62,D_63,D_64,D_65,D_66,D_68,D_69,D_70,D_71,D_72,D_73,D_74,D_75,D_76,D_77,D_78,D_79,D_80,D_81,D_82,D_83,D_84,D_86,D_87,D_88,D_89,D_91,D_92,D_93,D_94,D_96,D_102,D_103,D_104,D_105,D_106,D_107,D_108,D_109,D_110,D_111,D_112,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,D_130,D_131,D_132,D_133,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,S_3,S_5,S_6,S_7,S_8,S_9,S_11,S_12,S_13,S_15,S_16,S_17,S_18,S_19,S_20,S_22,S_23,S_24,S_25,S_26,S_27,P_2,P_3,P_4,B_1,B_2,B_3,B_4,B_5,B_6,B_7,B_8,B_9,B_10,B_11,B_12,B_13,B_14,B_15,B_16,B_17,B_18,B_19,B_20,B_21,B_22,B_23,B_24,B_25,B_26,B_27,B_28,B_29,B_30,B_31,B_32,B_33,B_36,B_37,B_38,B_39,B_40,B_41,B_42,R_1,R_2,R_3,R_4,R_5,R_6,R_7,R_8,R_9,R_10,R_11,R_12,R_13,R_14,R_15,R_16,R_17,R_18,R_19,R_20,R_21,R_22,R_23,R_24,R_25,R_26,R_27,R_28
0,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-03-05,0.008376,0.007045,NaN,0.355097,0.381721,0.232541,0.749022,0.931925,0.636798,NaN,NaN,0.008505,0.257918,NaN,1.009888,0.678041,NaN,0.673155,0.279011,0.001999,0.845026,0.112573,CO,R,0.003677,NaN,6.0,0.002893,0.255467,0.013176,0.007019,NaN,0.362555,0.338296,NaN,NaN,0.002008,0.009383,0.406832,0.008304,NaN,0.005456,0.008112,0.009082,NaN,NaN,0.007533,0.009429,0.006783,0.004019,0.002943,0.001115,0.004340,0.001371,0.007033,NaN,NaN,0.002144,NaN,0.006973,NaN,NaN,1.003346,0.003928,1.0,0.464769,0.0,-1.0,0.461045,0.459714,0.0,0.727194,0.428788,0.009761,0.365854,0.008233,1.0,0.009708,1.005559,0.007325,0.007439,0.007083,NaN,0.008561,NaN,NaN,NaN,NaN,NaN,0.008365,0.008801,0.001185,NaN,0.006018,0.009393,0.003585,0.401315,0.002239,0.008523,0.327650,0.127948,NaN,0.048080,0.184423,0.005208,0.408499,0.006978,0.008038,0.003621,0.004491,0.001774,0.853806,0.131347,0.868075,0.972705,0.004930,0.007958,0.523405,0.480283,0.001637,0.006680,1.000940,0.009355,0.901268,0.020332,0.034404,0.182016,0.007657,0.004749,0.027181,0.008233,0.154105,0.008163,0.006533,0.007149,0.423158,NaN,0.407278,0.109469,0.004621,0.005013,0.006692,0.184154,0.009886,0.003749,0.006558,0.004967,0.521413,NaN,0.0,1,0.000358,1.004676,0.000329,0.003749,3.0,NaN,0.330606,0.005743,NaN,0.003428,0.005751,0.507771,0.000728,0.008425,0.008175,0.002113,0.004826,NaN,0.007801,0.005049,1.000701,0.000934,0.001050,0.002937,0.005534,0.002551,0.009574,0.006862,0.007049,0.008520,0.005952,0.008763,0.000349,0.005358,NaN,1.000625,0.001186
1,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-04-10,0.002836,0.000565,NaN,0.312375,0.134045,0.232197,0.416163,0.939839,0.484000,NaN,0.006556,0.006998,0.253597,NaN,1.008957,0.611879,NaN,0.567915,0.275475,0.009932,0.842110,0.077126,CO,R,0.008699,NaN,6.0,0.005307,0.007638,0.012317,0.003940,NaN,0.290779,0.275103,NaN,0.072436,0.000811,0.009827,0.608082,0.008636,0.005899,0.009630,0.006017,0.001705,NaN,NaN,0.001632,0.001945,0.008371,0.002443,0.002520,0.005529,0.006890,0.009558,0.008599,NaN,NaN,0.008818,NaN,0.007328,NaN,NaN,1.002003,0.006697,1.0,0.477699,0.0,-1.0,0.461896,0.465308,0.0,0.732167,0.576645,0.004936,0.365156,0.008392,1.0,0.008002,1.002762,1.004064,0.004804,0.000823,NaN,0.000696,NaN,NaN,NaN,NaN,NaN,0.006429,0.000438,0.004144,NaN,0.002282,0.003261,0.002704,0.394459,0.004846,0.008937,0.322127,0.008656,NaN,0.289070,0.184495,0.004601,0.509170,0.002461,0.004669,0.004006,0.004215,0.004033,0.303755,0.140477,0.085234,0.969171,0.009193,0.005511,0.533376,0.522688,0.003317,0.007121,0.814569,0.011885,0.901686,0.009887,0.046754,0.328685,0.007138,0.008674,0.016825,0.002511,0.057497,0.012143,0.003472,0.004912,0.426391,NaN,0.407492,0.209688,0.009138,0.006126,0.005744,0.340210,0.002182,0.007864,0.005851,0.004641,0.505322,NaN,0.0,1,0.008105,1.005377,0.004619,0.003541,3.0,NaN,0.282520,0.008473,NaN,0.001009,0.005574,0.408810,0.005981,0.003779,0.000730,0.007040,0.003869,NaN,0.009881,0.007829,1.003468,0.009944,0.002582,0.000130,0.005474,0.006805,0.008637,0.000246,0.006

In [141]:
test_id = df['customer_ID'].unique()[:10].tolist()
train_id = df['customer_ID'].unique()[10:].tolist()

train = df[df['customer_ID'].isin(train_id)].reset_index(drop=True)
test = df[df['customer_ID'].isin(test_id)].reset_index(drop=True)

In [142]:
df.shape

(5569, 190)

In [143]:
train.shape

(5453, 190)

In [144]:
test.shape

(116, 190)

In [145]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
features = ['D_39', 'D_41', 'D_46', 'D_47']
# scaler.fit(train[features])

a = scaler.fit_transform(train[features])

In [146]:
a

array([[-0.54228466, -0.2248236 , -0.50905224, -0.21865219],
       [-0.54267629, -0.23478949, -0.38595656, -0.20829744],
       [-0.55000472, -0.24799928,  0.39942983, -0.23071497],
       ...,
       [-0.52788393, -0.25578756,  0.27817889,  1.47541242],
       [-0.52714526, -0.25547173,  0.03562294,  1.48065016],
       [-0.53664207, -0.22647093, -0.0318741 ,  1.49859812]])

In [147]:
a = pd.DataFrame(a)



In [148]:
a.isnull().sum()

0       0
1       3
2    1385
3       0
dtype: int64

In [151]:
train[features].isnull().sum()

D_39       0
D_41       3
D_46    1385
D_47       0
dtype: int64

In [154]:
features = ['D_39', 'D_41', 'D_46', 'D_47']

a = pd.DataFrame(train[features].isnull().sum(), columns=['train'])

a

,train
D_39,0
D_41,3
D_46,1385
D_47,0


In [137]:
train[features].head()

,D_39,D_41,D_46,D_47
0,0.002214,0.008778,0.395034,0.372449
1,0.002108,0.006091,0.415338,0.374858
2,0.000123,0.002530,0.544883,0.369643
3,0.001923,0.001291,0.416165,0.379595
4,0.008342,0.007556,0.426426,0.380488


In [163]:
null_df_train = pd.DataFrame()
null_df_test = pd.DataFrame()

for s in ['B', 'D', 'P', 'R', 'S']:
    with open(f'../input/train_data_prep_{s}.pkl', 'rb') as f:
        df = pickle.load(f)
    nulls = pd.DataFrame(df.isnull().sum(), columns=['train'])
    nulls['feature'] = df.isnull().sum().index
    null_df_train = pd.concat([null_df_train, nulls], axis=0, ignore_index=True)
    
    del nulls, df
    gc.collect()
        
for s in ['B', 'D', 'P', 'R', 'S']:
    with open(f'../input/test_data_prep_{s}.pkl', 'rb') as f:
        df = pickle.load(f)
    nulls = pd.DataFrame(df.isnull().sum(), columns=['test'])
    nulls['feature'] = df.isnull().sum().index
    null_df_test = pd.concat([null_df_test, nulls], axis=0, ignore_index=True)
    
    del nulls, df
    gc.collect()

In [165]:
null_df = pd.merge(null_df_train, null_df_test, on='feature')

In [166]:
# trainには欠損はないが、Testには欠損がある
null_df[(null_df['train'] == 0) &(null_df['test'] != 0)]

,train,feature,test
224,0,S_12,2385
228,0,S_17,12535


In [164]:
null_df_train.head()

,train,feature
0,0,customer_ID
1,0,S_2
2,0,B_1
3,2016,B_2
4,2016,B_3


In [295]:
# with open(f'../input/test_data_prep_D.pkl', 'rb') as f:
#     df = pickle.load(f)
# print(df.shape)

with open(f'../input/train_data_prep_D.pkl', 'rb') as f:
    train = pickle.load(f)
    
feats = ['D_39', 'D_41', 'D_42', 'D_43', 'D_44', 'D_45',
       'D_46', 'D_47', 'D_48', 'D_49', 'D_50', 'D_51', 'D_52', 'D_53', 'D_54',
       'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_61', 'D_62', 'D_63', 'D_64',
       'D_65', 'D_66', 'D_68', 'D_69', 'D_70', 'D_71', 'D_72', 'D_73', 'D_74',
       'D_75', 'D_76', 'D_77', 'D_78', 'D_79', 'D_80', 'D_81', 'D_82', 'D_83',
       'D_84', 'D_86', 'D_87', 'D_88', 'D_89', 'D_91', 'D_92', 'D_93', 'D_94',
       'D_96', 'D_102', 'D_103', 'D_104', 'D_105', 'D_106', 'D_107', 'D_108',
       'D_109', 'D_110', 'D_111', 'D_112', 'D_113', 'D_114', 'D_115', 'D_116',
       'D_117', 'D_118', 'D_119', 'D_120', 'D_121', 'D_122', 'D_123', 'D_124',
       'D_125', 'D_126', 'D_127', 'D_128', 'D_129', 'D_130', 'D_131', 'D_132',
       'D_133', 'D_134', 'D_135', 'D_136', 'D_137', 'D_138', 'D_139', 'D_140',
       'D_141', 'D_142', 'D_143', 'D_144', 'D_145']
train = train.groupby('customer_ID')[feats].last().reset_index()

# ラベルを付与
label = pd.read_csv('../input/train_labels.csv')
train = pd.merge(train, label, on='customer_ID', how='left')

In [296]:
train.shape

(458913, 98)

In [297]:
train.head()

,customer_ID,D_39,D_41,D_42,D_43,D_44,D_45,D_46,D_47,D_48,D_49,D_50,D_51,D_52,D_53,D_54,D_55,D_56,D_58,D_59,D_60,D_61,D_62,D_63,D_64,D_65,D_66,D_68,D_69,D_70,D_71,D_72,D_73,D_74,D_75,D_76,D_77,D_78,D_79,D_80,D_81,D_82,D_83,D_84,D_86,D_87,D_88,D_89,D_91,D_92,D_93,D_94,D_96,D_102,D_103,D_104,D_105,D_106,D_107,D_108,D_109,D_110,D_111,D_112,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,D_130,D_131,D_132,D_133,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0,0.001604,NaN,NaN,0,0.740234,0.420410,0.539551,0.192383,-1,0.153442,2,0.203491,NaN,1.007812,0.187256,0.166626,0.007175,8,0.258545,0.227661,0.446533,0,0,0,-1,6,0.005470,0,0.377930,0,NaN,0,0,NaN,0.421387,0,0,1,0,1,0,0,0,-1,NaN,0,2,0,0,0,0,0.943359,1,1.014648,1.074219,-1,2,-1,0,NaN,-1,1.009766,0,1,0.255859,0,5,0.260254,0.256592,0,0.719727,3,0,16,0,2,1,0.999512,1,0.004185,0.005703,NaN,0.006210,NaN,-1,-1,-1,-1,0,0,0.005085,NaN,0,0.002970,0,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,6,0.005550,NaN,0.060638,0,0.266357,0.438721,0.402100,0.014694,-1,NaN,1,0.242310,NaN,1.002930,0.036102,0.748535,0.009758,15,0.412109,0.048981,0.234009,3,0,0,-1,6,0.007637,0,0.007637,0,NaN,0,0,NaN,0.227783,0,0,0,0,-1,0,0,0,-1,NaN,0,0,0,0,0,0,0.001621,0,0.004364,NaN,-1,0,-1,0,NaN,-1,1.006836,0,1,0.454346,0,0,0.446045,0.436768,0,0.551270,2,0,4,0,2,0,0.999023,0,0.002201,0.001928,NaN,0.002996,NaN,-1,-1,-1,-1,0,0,0.007874,NaN,0,0.003170,0,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0,0.003796,NaN,NaN,0,0.251709,0.433594,0.339111,0.080383,-1,NaN,1,0.202148,NaN,1.005859,0.098938,0.209351,0.002848,15,0.002819,0.137817,0.438721,3,2,0,-1,6,0.009483,0,0.015022,0,NaN,0,0,NaN,0.407227,0,0,0,0,-1,0,0,0,-1,NaN,0,0,0,0,0,0,0.009789,0,0.005432,NaN,-1,0,-1,0,NaN,-1,1.006836,0,1,0.388672,0,0,0.368164,0.368896,0,0.444580,1,0,7,0,2,0,0.008331,0,0.002653,0.003469,NaN,0.009880,NaN,-1,-1,-1,-1,0,0,0.000965,NaN,0,0.000834,0,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0,0.004532,NaN,0.046112,0,0.085083,0.410645,0.414307,0.013054,-1,0.430420,1,0.198364,0.001379,1.003906,0.021393,0.554688,0.009293,29,0.394775,0.026840,0.434570,3,0,0,-1,3,0.004894,0,0.010681,0,NaN,0,0,NaN,0.401123,0,0,0,0,-1,0,0,0,-1,NaN,0,1,0,0,0,0,0.002447,0,0.004051,NaN,-1,0,-1,0,NaN,-1,1.003906,1,1,0.059357,0,7,0.058777,0.062744,0,0.628906,2,0,14,0,2,0,1.002930,1,0.000060,0.004578,NaN,0.001789,NaN,-1,-1,-1,-1,0,0,0.001899,NaN,0,0.005558,0,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0,0.000231,NaN,0.044678,0,0.069946,0.465576,0.480225,0.325195,-1,0.095215,0,0.253906,NaN,1.006836,0.254150,0.183105,0.382812,28,0.002670,0.600586,0.422852,3,0,0,1,6,0.008179,0,0.016464,0,NaN,2,2,NaN,0.392578,0,0,2,0,-1,0,0,0,-1,NaN,0,0,0,0,0,0,0.589355,1,0.963379,0.809570,-1,3,-1,0,NaN,-1,1.007812,0,1,0.448486,0,5,0.442383,0.436279,0,0.565918,3,0,5,0,2,0,1.005859,1,1.005859,0.008896,NaN,0.005047,NaN,-1,-1,-1,-1,0,0,0.005764,NaN,0,0.006943,0,0


In [299]:
train['customer_ID'] = train['customer_ID'].astype('category')

In [298]:
tar_cols = df.select_dtypes(include=[np.int8, np.int16, 'category']).columns

In [263]:
tar_cols = tar_cols.drop('customer_ID')

In [300]:
k = 3
f = 0.25

for c in ['customer_ID']:
    
    group = train.groupby(c)['target'].agg(['mean', 'count'])
    group = pd.DataFrame(group).reset_index()
    
    group['dataset_mean'] = train['target'].mean()
    
    group['lambda'] = group['count'].apply(lambda x: 1 / (1 + np.exp(-(x - k)/f)))
    group['target_enc'] = group['lambda'] * group['mean'] + (1 - group['lambda']) * group['dataset_mean']
    
    break

In [302]:
rep_dict = {k: v for k, v in zip(group['customer_ID'], group['target_enc'])}

train['tet'] = train['customer_ID'].map(rep_dict)

In [303]:
train.dtypes

customer_ID    category
D_39              int16
D_41            float16
D_42            float16
D_43            float16
D_44               int8
D_45            float16
D_46            float16
D_47            float16
D_48            float16
D_49              int16
D_50            float16
D_51               int8
D_52            float16
D_53            float16
D_54            float16
D_55            float16
D_56            float16
D_58            float16
D_59               int8
D_60            float16
D_61            float16
D_62            float16
D_63               int8
D_64               int8
D_65              int16
D_66               int8
D_68               int8
D_69            float16
D_70               int8
D_71            float16
D_72               int8
D_73            float16
D_74               int8
D_75               int8
D_76            float16
D_77            float16
D_78               int8
D_79               int8
D_80               int8
D_81               int8
D_82            

In [290]:
train['tar_enc_D_39'] = train['D_39'].map(enc_dict)

In [291]:
train[['D_39', 'tar_enc_D_39']].head(20)

,D_39,tar_enc_D_39
0,0,0.146680
1,6,0.262500
2,0,0.146680
3,0,0.146680
4,0,0.146680
5,0,0.146680
6,0,0.146680
7,0,0.146680
8,18,0.298218
9,0,0.146680


In [292]:
enc_dict

{0: 0.1466802817488434,
 1: 0.43831222580912177,
 2: 0.24623628476652207,
 3: 0.24427108727450025,
 4: 0.25433379729216754,
 5: 0.2443143129511099,
 6: 0.2625,
 7: 0.2555642838439382,
 8: 0.23929937091402492,
 9: 0.2580322718834785,
 10: 0.25831622176591373,
 11: 0.257256687535572,
 12: 0.26244343891402716,
 13: 0.2666463970811797,
 14: 0.2642447995176364,
 15: 0.2726238830219334,
 16: 0.2889856169568509,
 17: 0.2860055607043559,
 18: 0.29821750450630885,
 19: 0.2972408026755853,
 20: 0.2851883932438285,
 21: 0.32436781609195403,
 22: 0.32027300303336703,
 23: 0.33199381761978364,
 24: 0.36519114688128773,
 25: 0.4505431675242996,
 26: 0.5065217391304347,
 27: 0.5535499398315282,
 28: 0.561119293078056,
 29: 0.5938037865748709,
 30: 0.5984990619136961,
 31: 0.8566341334740174,
 32: 0.6614481409001957,
 33: 0.6797385620915033,
 34: 0.6914498141263941,
 35: 0.7530364372469636,
 36: 0.7154811715481172,
 37: 0.7457627118644068,
 38: 0.7432432432432432,
 39: 0.7401960784313726,
 40: 0.74444

In [190]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaler.fit(df[['S_12', 'S_17']])

StandardScaler()

In [191]:
scaler.mean_

array([0.24121141, 0.02032663])

In [196]:
a = np.array([0, np.nan, 99])

In [197]:
a

array([ 0., nan, 99.])

In [201]:
np.where(np.isnan(a), 0, a)



array([ 0.,  0., 99.])

In [194]:
mean_dict = {k: v for k, v in zip(['S_12', 'S_17'], scaler.mean_)}

In [195]:
mean_dict

{'S_12': 0.24121141233182042, 'S_17': 0.020326632599320923}

In [182]:
train[['S_12', 'S_17']].mean(axis=0)

/opt/conda/envs/rapids/lib/python3.8/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


S_12   NaN
S_17   NaN
dtype: float16

In [176]:
np.nanmean(df['S_12'])

/opt/conda/envs/rapids/lib/python3.8/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


inf

In [177]:
np.nanmean(df['S_17'])

inf

In [185]:
train['S_12'].describe()

count    5.531451e+06
mean              NaN
std      0.000000e+00
min     -4.040527e-01
25%      1.868896e-01
50%      1.907959e-01
75%      2.088623e-01
max      1.753750e+02
Name: S_12, dtype: float64

In [178]:
df[['S_12', 'S_17']].dtypes

S_12    float16
S_17    float16
dtype: object

In [179]:
df[df['S_12'] == np.inf]

,customer_ID,S_2,S_3,S_5,S_6,S_7,S_8,S_9,S_11,S_12,S_13,S_15,S_16,S_17,S_18,S_19,S_20,S_22,S_23,S_24,S_25,S_26,S_27


In [124]:
def get_last_values(df, features):
    group = df.groupby('customer_ID')[features].agg('last').reset_index()
    rename_dict = {k: f"fe_group_last_{k}" for k in features}
    group = group.rename(columns=rename_dict)
    
    return group

In [125]:
features = ['D_39', 'D_41', 'D_46', 'D_47']
train = get_last_values(train, features)
test = get_last_values(test, features)

In [126]:
from sklearn.decomposition import PCA

In [128]:
feats = [c for c in train.columns if c.startswith('fe')]
pca = PCA(n_components=6, random_state=0)

pca.fit(train[feats].values)

ValueError: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [118]:
train[features]

,customer_ID,fe_group_last_D_39,fe_group_last_D_41,fe_group_last_D_46,fe_group_last_D_47
0,074397d68f6828690b857f48252fd66274d002417cf153...,0.652277,0.253158,0.473553,0.386415
1,076fd580edb232321075bcdf3ef540dabc02f7d79248c1...,0.215735,0.001176,NaN,0.654144
2,07970dc9e83c621aabffe49c8bd17b1b87657e45cb5ca3...,0.000559,0.007794,0.413593,0.687767
3,0970253b4fa0f0a1d9732e43abe9d6780234e8b326f617...,0.003091,0.009901,0.515138,0.602375
4,097279b1669783e74e415a80737fdb34aeebefea3fab68...,0.064213,0.000332,0.542140,0.447474


In [32]:
window = 5
_df = df[['customer_ID', 'S_2', 'D_39']].copy()

_df = cudf.from_pandas(_df)

def func(x):
    n = 0
    for i in range(window):
        n += x.shift(i) / window
    return n

# func = lambda x: (x.shift(0) + x.shift(1) + x.shift(2)) / 3

_df['feats_D_39'] = _df.groupby('customer_ID')['D_39'].pipe(func)

In [33]:
_df.head()

,customer_ID,S_2,D_39,feats_D_39
0,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-03-05,0.008376,<NA>
1,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-04-10,0.002836,<NA>
2,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-05-11,0.005452,<NA>
3,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-06-26,0.008479,<NA>
4,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-07-27,0.001563,0.005341076


In [9]:
(0.008479 + 0.001563 + 0.005452)/3

0.005164666666666667

In [4]:
# df.groupby(['customer_ID']).rolling(window=2)['D_39'].mean().reset_index()
df['fe_rollingMean_D_39'] = df.groupby('customer_ID')['D_39'].transform(lambda x: x.rolling(2).mean())

_df = df.groupby('customer_ID').tail(1).reset_index(drop=True)

In [5]:
_df[['customer_ID', 'S_2', 'D_39', 'fe_rollingMean_D_39']].head()

,customer_ID,S_2,D_39,fe_rollingMean_D_39
0,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2018-03-29,0.007839,0.006902
1,00a0778eb2980d0fa97a0a7a4b32bb534d00437d30693b...,2018-03-10,0.002275,0.001970
2,0229ecb6a6eb8a724637bd7d8b988d06562f2441f5112e...,2018-03-20,0.000656,0.121753
3,0264ec10c09b97662a7b19871e4eaf540050afccf65d26...,2018-03-19,0.007139,0.004190
4,02f887d9280480f14f8cfe1650661f91b034f087f056b8...,2018-03-21,0.240051,0.461136


In [6]:
feats = ['D_39', 'D_41']

rename_list = [f'fe_rollingMean_{f}' for f in feats]

df[rename_list] = df.groupby('customer_ID')[feats].transform(lambda x: x.rolling(2).mean())

_df = df.groupby('customer_ID').tail(1).reset_index(drop=True)

In [7]:
df[rename_list].head()

,fe_rollingMean_D_39,fe_rollingMean_D_41
0,NaN,NaN
1,0.005606,0.003805
2,0.004144,0.002549
3,0.006965,0.004968
4,0.005021,0.003781


In [8]:
kmeans = KMeans(n_clusters=8)

In [19]:
feats = ['R_3', 'R_1']

kmeans = kmeans.fit(df[feats])

a = kmeans.predict(df[feats])
b = kmeans.transform(df[feats])

In [20]:
a

array([3, 3, 3, ..., 0, 0, 0], dtype=int32)

In [22]:
b[0]

array([0.47521348, 0.55065966, 1.10782913, 0.23212067, 0.6740886 ,
       0.29785597, 1.64150931, 1.25625403])

In [23]:
b = pd.DataFrame(b, columns=[np.arange(8)])

In [25]:
b.head()

,0,1,2,3,4,5,6,7
0,0.475213,0.550660,1.107829,0.232121,0.674089,0.297856,1.641509,1.256254
1,0.376319,0.579694,1.078037,0.133475,0.619393,0.396716,1.629673,1.349918
2,0.174069,0.677728,1.039569,0.070442,0.543327,0.598833,1.619611,1.542247
3,0.174772,0.676316,1.038462,0.069492,0.542336,0.598012,1.618415,1.541164
4,0.168899,0.678343,1.036094,0.074987,0.539316,0.603725,1.616667,1.546176


In [26]:
b.shape

(5569, 8)